### Import pandas library

In [156]:
import pandas as pd

### Read recipe json (populated from ChatGPT)

In [157]:
recipes = pd.read_json('../static/recipes.json', orient='records')
recipes = pd.json_normalize(recipes['recipes'], max_level=0)[['name', 'author', 'meal', 'dish', 'cuisine', 'servings', 'time_required', 'ingredients', 'directions']]
recipes = recipes.reset_index().rename(columns={'index': 'recipe_id'})

### Retrieve users from the recipe list to populate User table

In [158]:
users = recipes.drop_duplicates('author')['author'].reset_index(drop=True)
users = users.reset_index().rename(columns={'index': 'user_id', 'author': 'name'})

### Retrieve ingredients from the recipe list to populate Ingredient table

In [159]:
ingredients = recipes.explode('ingredients')
ingredients = ingredients.reset_index()
ingredients = ingredients.drop(columns='name')
split = pd.DataFrame(ingredients['ingredients'].to_list(), columns = ['name', 'amount', 'metric'])
ingredients = pd.concat([ingredients, split], axis=1)[['recipe_id', 'name', 'amount', 'metric']]
ingredients = ingredients.reset_index().rename(columns={'index': 'ingredient_id'})

### Use IDs from Ingredient and User table to add references in Recipe table

In [160]:
recipes['ingredients'] = recipes['recipe_id'].apply(lambda x: ingredients.loc[ingredients['recipe_id'] == x, 'ingredient_id'].to_list())

In [161]:
recipes['user_id'] = recipes['author'].apply(lambda x: users.loc[users['name']== x, 'user_id'].iloc[0])
recipes = recipes.drop(columns = 'author')

### Verify tables have been set up properly

In [162]:
recipes.head()

,recipe_id,name,meal,dish,cuisine,servings,time_required,ingredients,directions,user_id
0,0,Vibrant Green Detox Smoothie,Breakfast,Smoothie,Healthy,1,10 minutes,"[0, 1, 2, 3, 4, 5, 6, 7, 8]","[In a blender, combine spinach, kale, frozen b...",0
1,1,Quinoa and Chickpea Buddha Bowl,Main,Bowl,Mediterranean,2,30 minutes,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...",[Rinse the quinoa thoroughly and cook accordin...,0
2,2,Chocolate Avocado Protein Balls,Snack,Protein Balls,Healthy,12,20 minutes,"[22, 23, 24, 25, 26, 27, 28, 29, 30, 31]","[In a food processor, combine ripe avocado, oa...",0
3,3,Classic Beef Lasagna,Dinner,Pasta,Italian,6,1 hour 30 minutes,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4...","[Preheat the oven to 190°C (375°F)., In a skil...",1
4,4,Lemon Herb Roasted Chicken,Dinner,Roasted Chicken,American,4,1 hour 30 minutes,"[46, 47, 48, 49, 50, 51, 52, 53]","[Preheat the oven to 220°C (425°F)., Rinse the...",2


### QA/QC to make sure the numbers are isolated

In [163]:
ingredients['amount'].unique()

array(['2', '1', '1 can', '1 head', '3', '', '1/2', '1/4', '9', '24',
       '14.5', '15', '4', '6', '1 pound', '28 ounces', '6 ounces',
       '2 teaspoons', '1 teaspoon', '1/2 teaspoon', '15 ounces', '2 cups',
       '1/2 cup', '2 tablespoons', '14 ounces', '1 tablespoon', '1/4 cup',
       '4 cups', '1/4 teaspoon', '250', '200', 'a handful', '500', '8',
       'for dipping', '1 1/2', '400', 'for topping', '300', '100', '800',
       'for frying'], dtype=object)